In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import gmaps
import os
import scipy.stats as st
from scipy.stats import linregress


# Import keys
from config import w_api_key
from config import gkey

# Import citipy module
# Be sure to be in PythonData where this installed to have this work
from citipy import citipy

In [2]:
weather_data = pd.read_csv("../WeatherPy/City_Weather_data.csv")
weather_data

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,68.00,52,0,11.50,CL,1612819286
1,bredasdorp,-34.5322,20.0403,62.60,88,5,5.75,ZA,1612819481
2,cidreira,-30.1811,-50.2056,74.77,71,0,12.21,BR,1612819184
3,mataura,-46.1927,168.8643,60.39,85,100,2.33,NZ,1612819485
4,ponta do sol,32.6667,-17.1000,57.99,69,24,3.00,PT,1612819390
...,...,...,...,...,...,...,...,...,...
505,camalu,30.8500,-116.0667,64.00,43,0,5.99,MX,1612820091
506,praia da vitoria,38.7333,-27.0667,60.80,77,40,13.00,PT,1612819766
507,kihei,20.7850,-156.4656,77.00,61,40,19.57,US,1612820093
508,constantine,36.3650,6.6147,50.00,76,75,20.71,DZ,1612820094


In [3]:
# Configure google heatmap
gmaps.configure(api_key=gkey)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Long"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

# Dimensions seem a bit weird, but basic function seems to be working

Figure(layout=FigureLayout(height='420px'))

In [6]:
# What are my preferred vacations conditions?
# 60-80 degrees
# Average cloudiness, 45-70??
# Humidty 50-65%

In [7]:
# Narrow down the dataframe to ideal conditions
ideal_weather = weather_data.loc[weather_data["Humidity"] >= 30]
ideal_weather = ideal_weather.loc[weather_data["Humidity"] <= 70]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] >= 55]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] <= 85]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] >= 45]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] <= 75]

ideal_weather["Nearby Hotel"] = ""

ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
95,alice,27.7523,-98.0697,80.60,61,75,11.50,US,1612819595,
160,cayenne,4.9333,-52.3333,82.40,69,75,12.66,GF,1612819673,
206,nishihara,26.1842,127.7558,60.80,63,75,10.36,JP,1612819727,
227,avera,33.1940,-82.5271,60.80,59,75,12.66,US,1612819357,
307,los llanos de aridane,28.6585,-17.9182,66.20,63,75,1.14,ES,1612819761,
325,cabo rojo,18.0866,-67.1457,82.00,70,58,4.00,PR,1612819876,
331,fethiye,36.6217,29.1164,64.40,63,75,18.41,TR,1612819883,
351,rincon,32.2960,-81.2354,62.60,63,75,14.97,US,1612819906,
356,mar del plata,-38.0023,-57.5575,66.99,68,75,17.27,AR,1612819292,
416,huarmey,-10.0681,-78.1522,75.60,69,65,9.62,PE,1612819856,


In [10]:
# Let's check we're talking to google

waycross = "31.2136, -82.3540"
search_type = "hotel"
radius = 5000


params = {
    "location": waycross,
    "radius": radius,
    "types": search_type,
    "key": gkey
}

# "location": waycross as test case

In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()
response["results"][1]["name"]
# Ok! Well, it is basically working.  So that's a good sign

'Super 8 by Wyndham Waycross GA'

In [12]:
# Find the first hotel within 5000 meters of each city

search_type = "hotel"
radius = 5000

params = {
    "radius": radius,
    "types": search_type,
    "key": gkey
}

for index, row in ideal_weather.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_returns = requests.get(base_url, params=params)
    
    hotel_returns = hotel_returns.json()
    
    print("Retrieving next hotel")
    
    try:
        ideal_weather.loc[index, "Nearby Hotel"] = hotel_returns["results"][1]["name"]
        
    except (KeyError, IndexError):
        print("No Hotel Available")
        ideal_weather.loc[index, "Nearby Hotel"] = np.nan
        pass
    
    time.sleep(1)
    

Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel
Retrieving next hotel


In [13]:
ideal_weather.dropna(inplace=True)
ideal_weather.reset_index(drop=True, inplace=True)
ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
0,alice,27.7523,-98.0697,80.60,61,75,11.50,US,1612819595,Holiday Inn Express & Suites Alice
1,cayenne,4.9333,-52.3333,82.40,69,75,12.66,GF,1612819673,Hôtel Le Dronmi
2,nishihara,26.1842,127.7558,60.80,63,75,10.36,JP,1612819727,JEF
3,avera,33.1940,-82.5271,60.80,59,75,12.66,US,1612819357,"Hiram Bobo III, PC"
4,los llanos de aridane,28.6585,-17.9182,66.20,63,75,1.14,ES,1612819761,Valle Aridane
5,cabo rojo,18.0866,-67.1457,82.00,70,58,4.00,PR,1612819876,Hotel Perichi's
6,fethiye,36.6217,29.1164,64.40,63,75,18.41,TR,1612819883,Club & Hotel Letoonia
7,rincon,32.2960,-81.2354,62.60,63,75,14.97,US,1612819906,Subway
8,mar del plata,-38.0023,-57.5575,66.99,68,75,17.27,AR,1612819292,Gran Hotel Mar del Plata
9,huarmey,-10.0681,-78.1522,75.60,69,65,9.62,PE,1612819856,Hotel Miramar


In [21]:
locations_2 = ideal_weather[["Lat", "Long"]].astype(float)
humidity_2 = ideal_weather["Humidity"].astype(float)

info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]

fig_2 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_2, weights=humidity_2, 
                                 dissipating=False, max_intensity=70,
                                 point_radius = 1)

markers = gmaps.marker_layer(locations_2, info_box_content=info_box_template)

fig_2.add_layer(heat_layer)
fig_2.add_layer(markers)

fig_2

Figure(layout=FigureLayout(height='420px'))

In [ ]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Long"]] 

In [ ]:
# # Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=info_box_template)
fig.add_layers(markers)
fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap


marker_locations = ideal_weather[['Lat', 'Long']]
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearby Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Long"]]


In [ ]:
# # Plot markers on the map
# # Create a map using state centroid coordinates to set markers
# marker_locations = ideal_weather[['Lat', 'Long']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#                             info_box_content=info_box_template)
# fig.add_layer(markers)
# fig
